In [84]:
import numpy as np
from pathlib import Path    

data_path = Path("..") / "data" / "point_cloud_20260204T140758Z.npz"
data = np.load(data_path)

print(data.files)

['points', 'width', 'height', 'depth_scale', 'fx', 'fy', 'cx', 'cy', 'intr_width', 'intr_height']


In [85]:
import numpy as np
import plotly.graph_objects as go

# Use the points array from the loaded npz
points = data["points"] if "points" in data.files else data[data.files[0]]

# Clean and optionally downsample for responsiveness
mask = np.isfinite(points).all(axis=1)
pts = points[mask]
pts = pts[~np.all(pts == 0, axis=1)]

# max_points = 200000
# if pts.shape[0] > max_points:
#     rng = np.random.default_rng(0)
#     idx = rng.choice(pts.shape[0], max_points, replace=False)
#     pts = pts[idx]

# fig = go.Figure(data=[go.Scatter3d(
#     x=pts[:, 0], y=pts[:, 1], z=pts[:, 2],
#     mode='markers',
#     marker=dict(size=1, color=pts[:, 2], colorscale='Viridis', opacity=0.7)
# )])
# fig.update_layout(
#     title='Point Cloud',
#     scene=dict(aspectmode='data'),
#     margin=dict(l=0, r=0, b=0, t=30)
# )
# fig.show()


In [86]:
from dataclasses import dataclass

@dataclass
class DimsAlgoConfig:
    # --- point cloud preprocessing ---
    voxel_size: float = 0.001          # 3 мм
    nb_neighbors: int = 20
    std_ratio: float = 5.0

    # --- plane (table) ---
    plane_dist_thresh: float = 0.004   # 4 мм: допуск точек к плоскости
    ransac_n: int = 3
    ransac_iters: int = 1000

    # --- object extraction relative to table ---
    h_min: float = 0.003               # выше стола минимум 3 мм (чтобы не цеплять стол)
    h_max: float = 1.0                 # максимум 1 м (защита от мусора)
    
    # ROI в координатах стола (метры), если известны габариты рабочего поля
    roi_x_min: float = -1.40
    roi_x_max: float =  1.40
    roi_y_min: float = -1.30
    roi_y_max: float =  1.30

    # --- clustering ---
    use_dbscan: bool = True
    dbscan_eps: float = 0.01           # 1 см
    dbscan_min_points: int = 30

    # --- robust extents ---
    q_low: float = 0.01
    q_high: float = 0.99

In [87]:
# preprocessing

import open3d as o3d

o3d_points = o3d.utility.Vector3dVector(points)
pcd = o3d.geometry.PointCloud(o3d_points)
cfg = DimsAlgoConfig(voxel_size= 1,
                     nb_neighbors= 100,
                     std_ratio= 1.25)

if cfg.voxel_size > 0:
    pcd = pcd.voxel_down_sample(cfg.voxel_size)
pcd, _ = pcd.remove_statistical_outlier(
    nb_neighbors=cfg.nb_neighbors,
    std_ratio=cfg.std_ratio
)

o3d.visualization.draw_geometries([pcd])


In [88]:
# table plane estimation

cfg = DimsAlgoConfig(plane_dist_thresh = 5.0,
                     ransac_n = 3,
                     ransac_iters = 1000)
plane_model, inliers = pcd.segment_plane(
    distance_threshold=cfg.plane_dist_thresh,
    ransac_n=cfg.ransac_n,
    num_iterations=cfg.ransac_iters
)

inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)

o3d.visualization.draw_geometries([outlier_cloud])


In [98]:
# signed distance

a, b, c, d = plane_model
n = np.array([a, b, c], dtype=np.float64)
nn = np.linalg.norm(n)

n = n/nn
d = d/nn

if d<0:
    n = -n
    d = -d

sd_pts = np.asarray(outlier_cloud.points)
sd = sd_pts @ n + d

obj_pts = sd_pts[sd>3]

filtered_points = o3d.utility.Vector3dVector(obj_pts)
filtered_pcd = o3d.geometry.PointCloud(filtered_points)

o3d.visualization.draw_geometries([filtered_pcd])


In [ ]:
import numpy as np

def _normalize(v: np.ndarray) -> np.ndarray:
    n = np.linalg.norm(v)
    if n < 1e-12:
        raise ValueError("Zero vector normalization")
    return v / n

def make_table_frame(plane_model):
    """
    plane_model: (a,b,c,d) for ax+by+cz+d=0
    Returns:
      R: (3,3) rotation matrix with columns [X_table, Y_table, Z_table] in camera frame
      p0: point on plane in camera frame (origin of table frame)
      n: unit normal (Z_table) in camera frame
    """
    a, b, c, d = plane_model
    n = _normalize(np.array([a, b, c], dtype=np.float64))  # Z_table in camera coords
    
    # point on plane: for unit normal, p0 = -d * n
    p0 = -d * n

    # choose a stable reference vector not parallel to n
    ref = np.array([1.0, 0.0, 0.0])
    if abs(np.dot(ref, n)) > 0.9:
        ref = np.array([0.0, 1.0, 0.0])

    x = _normalize(ref - np.dot(ref, n) * n)
    y = _normalize(np.cross(n, x))

    R = np.column_stack([x, y, n])  # columns are axes in camera frame
    return R, p0, n

def transform_cam_to_table(points_xyz: np.ndarray, R: np.ndarray, p0: np.ndarray) -> np.ndarray:
    """
    points_table = R^T (points_cam - p0)
    """
    return (R.T @ (points_xyz - p0).T).T